# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff97098ed30>
├── 'a' --> tensor([[-0.6042, -0.6698, -1.1385],
│                   [-1.0516, -1.4043, -0.1458]])
└── 'x' --> <FastTreeValue 0x7ff97098ebe0>
    └── 'c' --> tensor([[ 0.5048,  0.6089, -2.0348,  1.0639],
                        [ 0.3563,  0.7882,  1.1929, -1.0652],
                        [ 0.8315,  0.1529,  1.0504, -0.6659]])

In [4]:
t.a

tensor([[-0.6042, -0.6698, -1.1385],
        [-1.0516, -1.4043, -0.1458]])

In [5]:
%timeit t.a

46.6 ns ± 0.9 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff97098ed30>
├── 'a' --> tensor([[-0.3939,  0.9285, -0.0783],
│                   [ 0.5034,  0.0010, -0.4377]])
└── 'x' --> <FastTreeValue 0x7ff97098ebe0>
    └── 'c' --> tensor([[ 0.5048,  0.6089, -2.0348,  1.0639],
                        [ 0.3563,  0.7882,  1.1929, -1.0652],
                        [ 0.8315,  0.1529,  1.0504, -0.6659]])

In [7]:
%timeit t.a = new_value

50.2 ns ± 0.582 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6042, -0.6698, -1.1385],
               [-1.0516, -1.4043, -0.1458]]),
    x: Batch(
           c: tensor([[ 0.5048,  0.6089, -2.0348,  1.0639],
                      [ 0.3563,  0.7882,  1.1929, -1.0652],
                      [ 0.8315,  0.1529,  1.0504, -0.6659]]),
       ),
)

In [10]:
b.a

tensor([[-0.6042, -0.6698, -1.1385],
        [-1.0516, -1.4043, -0.1458]])

In [11]:
%timeit b.a

41 ns ± 0.293 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.0806,  0.8120, -0.5090],
               [-1.1575, -0.1895, -0.6689]]),
    x: Batch(
           c: tensor([[ 0.5048,  0.6089, -2.0348,  1.0639],
                      [ 0.3563,  0.7882,  1.1929, -1.0652],
                      [ 0.8315,  0.1529,  1.0504, -0.6659]]),
       ),
)

In [13]:
%timeit b.a = new_value

366 ns ± 3.48 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

658 ns ± 0.998 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

8.56 µs ± 49.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

132 µs ± 476 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

129 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff879d763d0>
├── 'a' --> tensor([[[-0.6042, -0.6698, -1.1385],
│                    [-1.0516, -1.4043, -0.1458]],
│           
│                   [[-0.6042, -0.6698, -1.1385],
│                    [-1.0516, -1.4043, -0.1458]],
│           
│                   [[-0.6042, -0.6698, -1.1385],
│                    [-1.0516, -1.4043, -0.1458]],
│           
│                   [[-0.6042, -0.6698, -1.1385],
│                    [-1.0516, -1.4043, -0.1458]],
│           
│                   [[-0.6042, -0.6698, -1.1385],
│                    [-1.0516, -1.4043, -0.1458]],
│           
│                   [[-0.6042, -0.6698, -1.1385],
│                    [-1.0516, -1.4043, -0.1458]],
│           
│                   [[-0.6042, -0.6698, -1.1385],
│                    [-1.0516, -1.4043, -0.1458]],
│           
│                   [[-0.6042, -0.6698, -1.1385],
│                    [-1.0516, -1.4043, -0.1458]]])
└── 'x' --> <FastTreeValue 0x7ff879d766d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

24.4 µs ± 346 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff97098ed60>
├── 'a' --> tensor([[-0.6042, -0.6698, -1.1385],
│                   [-1.0516, -1.4043, -0.1458],
│                   [-0.6042, -0.6698, -1.1385],
│                   [-1.0516, -1.4043, -0.1458],
│                   [-0.6042, -0.6698, -1.1385],
│                   [-1.0516, -1.4043, -0.1458],
│                   [-0.6042, -0.6698, -1.1385],
│                   [-1.0516, -1.4043, -0.1458],
│                   [-0.6042, -0.6698, -1.1385],
│                   [-1.0516, -1.4043, -0.1458],
│                   [-0.6042, -0.6698, -1.1385],
│                   [-1.0516, -1.4043, -0.1458],
│                   [-0.6042, -0.6698, -1.1385],
│                   [-1.0516, -1.4043, -0.1458],
│                   [-0.6042, -0.6698, -1.1385],
│                   [-1.0516, -1.4043, -0.1458]])
└── 'x' --> <FastTreeValue 0x7ff97098efa0>
    └── 'c' --> tensor([[ 0.5048,  0.6089, -2.0348,  1.0639],
                        [ 0.3563,  0.7882,  1.1929, -1.0652],
                 

In [23]:
%timeit t_cat(trees)

22.3 µs ± 274 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

50.9 µs ± 404 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.6042, -0.6698, -1.1385],
                [-1.0516, -1.4043, -0.1458]],
       
               [[-0.6042, -0.6698, -1.1385],
                [-1.0516, -1.4043, -0.1458]],
       
               [[-0.6042, -0.6698, -1.1385],
                [-1.0516, -1.4043, -0.1458]],
       
               [[-0.6042, -0.6698, -1.1385],
                [-1.0516, -1.4043, -0.1458]],
       
               [[-0.6042, -0.6698, -1.1385],
                [-1.0516, -1.4043, -0.1458]],
       
               [[-0.6042, -0.6698, -1.1385],
                [-1.0516, -1.4043, -0.1458]],
       
               [[-0.6042, -0.6698, -1.1385],
                [-1.0516, -1.4043, -0.1458]],
       
               [[-0.6042, -0.6698, -1.1385],
                [-1.0516, -1.4043, -0.1458]]]),
    x: Batch(
           c: tensor([[[ 0.5048,  0.6089, -2.0348,  1.0639],
                       [ 0.3563,  0.7882,  1.1929, -1.0652],
                       [ 0.8315,  0.1529,  1.0504, -0.6659]],
         

In [26]:
%timeit Batch.stack(batches)

64.2 µs ± 339 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.6042, -0.6698, -1.1385],
               [-1.0516, -1.4043, -0.1458],
               [-0.6042, -0.6698, -1.1385],
               [-1.0516, -1.4043, -0.1458],
               [-0.6042, -0.6698, -1.1385],
               [-1.0516, -1.4043, -0.1458],
               [-0.6042, -0.6698, -1.1385],
               [-1.0516, -1.4043, -0.1458],
               [-0.6042, -0.6698, -1.1385],
               [-1.0516, -1.4043, -0.1458],
               [-0.6042, -0.6698, -1.1385],
               [-1.0516, -1.4043, -0.1458],
               [-0.6042, -0.6698, -1.1385],
               [-1.0516, -1.4043, -0.1458],
               [-0.6042, -0.6698, -1.1385],
               [-1.0516, -1.4043, -0.1458]]),
    x: Batch(
           c: tensor([[ 0.5048,  0.6089, -2.0348,  1.0639],
                      [ 0.3563,  0.7882,  1.1929, -1.0652],
                      [ 0.8315,  0.1529,  1.0504, -0.6659],
                      [ 0.5048,  0.6089, -2.0348,  1.0639],
                      [ 0.3563,  

In [28]:
%timeit Batch.cat(batches)

120 µs ± 711 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

287 µs ± 3.57 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
